In [372]:
import numpy as np

In [373]:
# Policy parameterization: Softmax
# State value parameterization: Linear
# State feature vector representation: Fourier (as in HW2)

In [374]:
M = 5 # dimensionality of the fourier transform [2, 10]
# mean_policy_vector = np.ones(2*M+1)*0.05 #some non-zero vector

policy_params = np.ones((2*M+1,2))*0.01
value_params = np.ones(2*M+1)*0.01

softmax_sigma = 0.1
# policy_params = np.zeros((np.power(M+1,2),2))
# value_params = np.zeros(np.power(M+1,2))

policy_step_size = 1e-3 #2e-9
valueFunction_step_size = 5e-4
gamma = 1

In [375]:
def phi_s_cosine(given_x, given_v, cosineFlag):
    normal_x = normalized_x(given_x, cosineFlag)
    normal_v = normalized_v(given_v, cosineFlag)
    feature_phi = np.zeros(2*M+1)
#     feature_phi = np.zeros(np.power(M+1,2))
    feature_phi[0] = 1
    if cosineFlag:
        for i in range(1, M+1):
            feature_phi[i] = np.cos(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.cos(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
    
    else:
        for i in range(1, M+1):
            feature_phi[i] = np.sin(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.sin(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))

    return feature_phi

In [376]:
def normalized_x(given_x, cosineFlag):
    if cosineFlag:
        return (given_x+1.2)/(0.5+1.2)
    else:
        return ((given_x+1.2)/(0.5+1.2))*2 - 1
def normalized_v(given_v, cosineFlag):
    if cosineFlag:
        return (given_v+0.7)/(0.7+0.7)
    else:
        return ((given_v+0.7)/(0.7+0.7))*2 - 1

In [377]:
def runEpisode(policy_params,value_params, gamma):
    
    episode_trace = []
    
    total_reward = 0
    x_0 = np.random.uniform(-0.6,-0.4,1)[0]
    begin_state = (x_0,0)
    
    S_t = begin_state
    S_t_plus_1 = begin_state
    for t in range(0, 1000):
        
        state_t_feature_vector = phi_s_cosine(S_t[0], S_t[1], True)
        state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        exp_terms = np.exp(softmax_sigma * state_t_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
#         A_t = np.random.choice([-1,1], p=list(pi_values))
        
        max_action_idx = np.argmax(pi_values)
        if max_action_idx==1:
            A_t = 1
        else:
            A_t = -1
        
#         threshold = np.dot(state_feature_vector, given_theta)
        
#         print(threshold)
        
#         if threshold<=0:
#             A_t = -1
#         else:
#             A_t = 1
        
        v_t_plus_1 = S_t[1] + 0.001*A_t - 0.0025*np.cos(3*S_t[0])
        x_t_plus_1 = S_t[0] + v_t_plus_1
        
        
        
        if x_t_plus_1 < -1.2:
            x_t_plus_1 = -1.2
            v_t_plus_1 = 0
        elif x_t_plus_1 > 0.5:
            x_t_plus_1 = 0.5
            v_t_plus_1 = 0
        
        if v_t_plus_1 < -0.7:
            v_t_plus_1 = -0.7
        elif v_t_plus_1 > 0.7:
            v_t_plus_1 = 0.7
        
        S_t_plus_1 = (x_t_plus_1, v_t_plus_1)
        
        current_reward = -1
        if x_t_plus_1 == 0.5:
            current_reward = 0
            print("Terminated")
            episode_trace.append((S_t, A_t, 0))
#             break
        else:
            episode_trace.append((S_t, A_t, -1))

            current_reward = -1

            total_reward += -1
#         print("After ",total_reward)
        
    
        v_hat_s = np.dot(state_t_feature_vector, value_params)
        v_hat_s_prime = np.dot(phi_s_cosine(S_t_plus_1[0], S_t_plus_1[1], True),value_params)
        if x_t_plus_1 == 0.5:
            v_hat_s_prime = 0
        
        delta = current_reward + gamma*v_hat_s_prime - v_hat_s
        
        value_params += valueFunction_step_size*delta*state_t_feature_vector
        

        state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        
        
        exp_terms = np.exp(softmax_sigma*state_t_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
#         print("Pi values: ",pi_values)
#         print("G: ",G,". Delta: ",delta)
        if A_t == -1:
            policy_params[:,0] += policy_step_size*delta*(1-pi_values[0])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(-pi_values[0])*state_t_feature_vector
        else:
            policy_params[:,0] += policy_step_size*delta*(-pi_values[1])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(1-pi_values[1])*state_t_feature_vector
    
    
        if x_t_plus_1 == 0.5:
            break

        S_t = S_t_plus_1 #Incrementing time step
    return episode_trace

In [378]:
#With baseline

curr_iter = 0
while(curr_iter<=5000):
    curr_iter += 1
    policy_params_temp = policy_params.copy()    
    episode_trace = runEpisode(policy_params,value_params, gamma)
    print("\n EPISODE LENGTH: ",len(episode_trace), "CURR ITER: ", curr_iter)

#     for a_step in range(len(episode_trace)):
#         G = 0
#         for idx in range(a_step, len(episode_trace)):
#             G += episode_trace[idx][2]
        
#         state_t = episode_trace[a_step][0]
#         state_t_feature_vector = phi_s_cosine(state_t[0], state_t[1], True)

#         v_hat_s = np.dot(state_t_feature_vector, value_params)
#         delta = G - v_hat_s
        
#         value_params += valueFunction_step_size*delta*state_t_feature_vector
        

#         state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        
        
#         exp_terms = np.exp(softmax_sigma*state_t_feature_policy_param_product)
#         pi_values = exp_terms/np.sum(exp_terms)
        
# #         print("Pi values: ",pi_values)
# #         print("G: ",G,". Delta: ",delta)
#         if episode_trace[a_step][1] == -1:
#             policy_params[:,0] += policy_step_size*delta*(1-pi_values[0])*state_t_feature_vector
#             policy_params[:,1] += policy_step_size*delta*(-pi_values[0])*state_t_feature_vector
#         else:
#             policy_params[:,0] += policy_step_size*delta*(-pi_values[1])*state_t_feature_vector
#             policy_params[:,1] += policy_step_size*delta*(1-pi_values[1])*state_t_feature_vector
    
    max_diff = np.max(np.abs(policy_params_temp - policy_params))
    print(policy_step_size," Max diff: ",max_diff)
    if max_diff<policy_step_size*1e-1 or len(episode_trace)<110:# 0.001 works with 1e-6 policy_step
        break


 EPISODE LENGTH:  1000 CURR ITER:  1
0.001  Max diff:  0.0009199983039362338

 EPISODE LENGTH:  1000 CURR ITER:  2
0.001  Max diff:  0.0008696872610353488

 EPISODE LENGTH:  1000 CURR ITER:  3
0.001  Max diff:  0.002814483881243215

 EPISODE LENGTH:  1000 CURR ITER:  4
0.001  Max diff:  0.0027625182974334436

 EPISODE LENGTH:  1000 CURR ITER:  5
0.001  Max diff:  0.003187103329343459

 EPISODE LENGTH:  1000 CURR ITER:  6
0.001  Max diff:  0.0010988661852275369

 EPISODE LENGTH:  1000 CURR ITER:  7
0.001  Max diff:  0.002197309505185801

 EPISODE LENGTH:  1000 CURR ITER:  8
0.001  Max diff:  0.002624539100603167

 EPISODE LENGTH:  1000 CURR ITER:  9
0.001  Max diff:  0.002643953531957048
Terminated

 EPISODE LENGTH:  405 CURR ITER:  10
0.001  Max diff:  0.008355511269305928
Terminated

 EPISODE LENGTH:  350 CURR ITER:  11
0.001  Max diff:  0.003978957682431672
Terminated

 EPISODE LENGTH:  306 CURR ITER:  12
0.001  Max diff:  0.0040376368402464495
Terminated

 EPISODE LENGTH:  348 CURR

Terminated

 EPISODE LENGTH:  201 CURR ITER:  97
0.001  Max diff:  0.02255011495730086
Terminated

 EPISODE LENGTH:  164 CURR ITER:  98
0.001  Max diff:  0.02196689898808668
Terminated

 EPISODE LENGTH:  150 CURR ITER:  99
0.001  Max diff:  0.022896446635964296
Terminated

 EPISODE LENGTH:  204 CURR ITER:  100
0.001  Max diff:  0.025633133765172178
Terminated

 EPISODE LENGTH:  201 CURR ITER:  101
0.001  Max diff:  0.023134523622276082
Terminated

 EPISODE LENGTH:  204 CURR ITER:  102
0.001  Max diff:  0.025113235420852642
Terminated

 EPISODE LENGTH:  154 CURR ITER:  103
0.001  Max diff:  0.024087520676492447
Terminated

 EPISODE LENGTH:  199 CURR ITER:  104
0.001  Max diff:  0.025441554819012002
Terminated

 EPISODE LENGTH:  170 CURR ITER:  105
0.001  Max diff:  0.02246718874287701
Terminated

 EPISODE LENGTH:  247 CURR ITER:  106
0.001  Max diff:  0.03388978906859097
Terminated

 EPISODE LENGTH:  189 CURR ITER:  107
0.001  Max diff:  0.019925038708065612
Terminated

 EPISODE LENGTH:

Terminated

 EPISODE LENGTH:  279 CURR ITER:  198
0.001  Max diff:  0.06471857591447261
Terminated

 EPISODE LENGTH:  228 CURR ITER:  199
0.001  Max diff:  0.058708426208225806
Terminated

 EPISODE LENGTH:  156 CURR ITER:  200
0.001  Max diff:  0.050032904665023636
Terminated

 EPISODE LENGTH:  211 CURR ITER:  201
0.001  Max diff:  0.05096118199922017
Terminated

 EPISODE LENGTH:  203 CURR ITER:  202
0.001  Max diff:  0.05204095110732432
Terminated

 EPISODE LENGTH:  159 CURR ITER:  203
0.001  Max diff:  0.04650322634281956
Terminated

 EPISODE LENGTH:  157 CURR ITER:  204
0.001  Max diff:  0.04209805417806933
Terminated

 EPISODE LENGTH:  279 CURR ITER:  205
0.001  Max diff:  0.06744253473365447
Terminated

 EPISODE LENGTH:  230 CURR ITER:  206
0.001  Max diff:  0.06380905087224642
Terminated

 EPISODE LENGTH:  158 CURR ITER:  207
0.001  Max diff:  0.05293092253450382
Terminated

 EPISODE LENGTH:  162 CURR ITER:  208
0.001  Max diff:  0.05005363479007485
Terminated

 EPISODE LENGTH:  

Terminated

 EPISODE LENGTH:  380 CURR ITER:  298
0.001  Max diff:  0.15004800506632954
Terminated

 EPISODE LENGTH:  457 CURR ITER:  299
0.001  Max diff:  0.15607139445435791
Terminated

 EPISODE LENGTH:  145 CURR ITER:  300
0.001  Max diff:  0.08168838139499623
Terminated

 EPISODE LENGTH:  234 CURR ITER:  301
0.001  Max diff:  0.08334470628765178
Terminated

 EPISODE LENGTH:  162 CURR ITER:  302
0.001  Max diff:  0.10934903302614352
Terminated

 EPISODE LENGTH:  159 CURR ITER:  303
0.001  Max diff:  0.07170900072329545
Terminated

 EPISODE LENGTH:  236 CURR ITER:  304
0.001  Max diff:  0.06886522888758595
Terminated

 EPISODE LENGTH:  251 CURR ITER:  305
0.001  Max diff:  0.09613788104210386
Terminated

 EPISODE LENGTH:  363 CURR ITER:  306
0.001  Max diff:  0.12190852040616917
Terminated

 EPISODE LENGTH:  384 CURR ITER:  307
0.001  Max diff:  0.15238207062315645
Terminated

 EPISODE LENGTH:  237 CURR ITER:  308
0.001  Max diff:  0.08528894564591738
Terminated

 EPISODE LENGTH:  43

Terminated

 EPISODE LENGTH:  547 CURR ITER:  396
0.001  Max diff:  0.24562659867407088
Terminated

 EPISODE LENGTH:  192 CURR ITER:  397
0.001  Max diff:  0.06090642051965567
Terminated

 EPISODE LENGTH:  460 CURR ITER:  398
0.001  Max diff:  0.15944785052027477
Terminated

 EPISODE LENGTH:  527 CURR ITER:  399
0.001  Max diff:  0.19544895942988205
Terminated

 EPISODE LENGTH:  622 CURR ITER:  400
0.001  Max diff:  0.2796314480077191
Terminated

 EPISODE LENGTH:  319 CURR ITER:  401
0.001  Max diff:  0.15630994078385996
Terminated

 EPISODE LENGTH:  388 CURR ITER:  402
0.001  Max diff:  0.11654619607893402
Terminated

 EPISODE LENGTH:  149 CURR ITER:  403
0.001  Max diff:  0.11497201976368387
Terminated

 EPISODE LENGTH:  255 CURR ITER:  404
0.001  Max diff:  0.0974861806785583
Terminated

 EPISODE LENGTH:  234 CURR ITER:  405
0.001  Max diff:  0.1338862662082354
Terminated

 EPISODE LENGTH:  180 CURR ITER:  406
0.001  Max diff:  0.15134461654042397
Terminated

 EPISODE LENGTH:  691 C